# Intents - Encoding SRE Expertise

This notebook demonstrates **INTENT** definitions in elastic-script. Intents encapsulate operational knowledge with:

- **DESCRIPTION** - What the intent does
- **REQUIRES** - Pre-conditions that must be true
- **ACTIONS** - Steps to execute
- **ON_FAILURE** - Error handling

Intents are designed for AI/LLM agents to safely execute operational tasks with guardrails.

## 1. Basic Intent Definition

In [ ]:
-- Define a simple intent with description and actions
DEFINE INTENT greet_user(name STRING)
DESCRIPTION 'Greets a user by name'
ACTIONS
    PRINT 'Hello, ' || name || '!';
    PRINT 'Welcome to elastic-script!';
END INTENT

In [ ]:
-- Invoke the intent with positional argument
INTENT greet_user('Alice');

In [ ]:
-- Invoke with named argument
INTENT greet_user WITH name = 'Bob';

## 2. Intent with REQUIRES Pre-conditions

In [ ]:
-- Define an intent with pre-conditions
DEFINE INTENT scale_replicas(replicas NUMBER)
DESCRIPTION 'Scale service to specified replicas'
REQUIRES
    replicas > 0,
    replicas <= 100
ACTIONS
    PRINT 'Scaling to ' || replicas || ' replicas...';
    PRINT 'Scale operation complete.';
END INTENT

In [ ]:
-- This will succeed (within bounds)
INTENT scale_replicas(5);

In [ ]:
-- This will fail REQUIRES (zero replicas)
INTENT scale_replicas(0);

## 3. Intent with ON_FAILURE Handling

In [ ]:
-- Define an intent with failure handling
DEFINE INTENT process_data(source STRING)
DESCRIPTION 'Processes data from a source'
ACTIONS
    PRINT 'Processing data from: ' || source;
    IF source = 'invalid' THEN
        RAISE 'Invalid data source';
    END IF;
    PRINT 'Data processed successfully.';
ON_FAILURE
    PRINT 'ERROR: ' || ERROR_MESSAGE;
    PRINT 'Initiating cleanup...';
END INTENT

In [ ]:
-- This will succeed
INTENT process_data('logs-sample');

In [ ]:
-- This will trigger ON_FAILURE
INTENT process_data('invalid');

## 4. Introspection - List All Intents

In [ ]:
-- List all defined intents
CREATE PROCEDURE list_intents()
BEGIN
    DECLARE intents ARRAY = ESCRIPT_INTENTS();
    
    PRINT '=== Defined Intents ===';
    FOR intent IN intents LOOP
        PRINT intent['signature'] || ' - ' || intent['description'];
    END LOOP;
END PROCEDURE;

CALL list_intents();

In [ ]:
-- Get details about a specific intent
CREATE PROCEDURE intent_details(intent_name STRING)
BEGIN
    DECLARE info DOCUMENT = ESCRIPT_INTENT(intent_name);
    
    PRINT '=== Intent: ' || intent_name || ' ===';
    PRINT 'Exists: ' || info['exists'];
    IF info['exists'] = TRUE THEN
        PRINT 'Signature: ' || info['signature'];
        PRINT 'Description: ' || info['description'];
        PRINT 'Has REQUIRES: ' || info['has_requires'];
        PRINT 'Has ON_FAILURE: ' || info['has_on_failure'];
    END IF;
END PROCEDURE;

CALL intent_details('scale_replicas');

## 5. Real-World Example: Service Health Check

In [ ]:
-- A more realistic intent for SRE operations
DEFINE INTENT check_service_health(service_name STRING, threshold NUMBER)
DESCRIPTION 'Check service health based on error rate threshold'
REQUIRES
    threshold > 0,
    threshold <= 100,
    LENGTH(service_name) > 0
ACTIONS
    PRINT 'Checking health for service: ' || service_name;
    PRINT 'Error threshold: ' || threshold || '%';
    
    -- Query logs for the service
    DECLARE logs ARRAY = ESQL_QUERY('
        FROM logs-sample
        | STATS error_count = COUNT(*) WHERE level = "ERROR",
                total_count = COUNT(*)
    ');
    
    IF ARRAY_LENGTH(logs) > 0 THEN
        DECLARE stats DOCUMENT = logs[0];
        DECLARE error_rate NUMBER = (stats["error_count"] / stats["total_count"]) * 100;
        PRINT 'Current error rate: ' || ROUND(error_rate, 2) || '%';
        
        IF error_rate > threshold THEN
            PRINT 'ALERT: Error rate exceeds threshold!';
            RAISE 'Service ' || service_name || ' is unhealthy';
        ELSE
            PRINT 'Service is healthy.';
        END IF;
    END IF;
ON_FAILURE
    PRINT 'Health check failed: ' || ERROR_MESSAGE;
    -- Could trigger alert here
    PRINT 'Consider investigating service: ' || service_name;
END INTENT

In [ ]:
-- Invoke the health check
INTENT check_service_health WITH service_name = 'api-gateway', threshold = 10;